In [ ]:
%%writefile submission.py

# Imports helper functions
from pprint import pprint
from random import *
from kaggle_environments.envs.halite.helpers import *

# Returns best direction to move from one position (fromPos) to another (toPos)
# Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?
def getDirTo(fromPos, toPos, size):
    fromX, fromY = divmod(fromPos[0],size), divmod(fromPos[1],size)
    toX, toY = divmod(toPos[0],size), divmod(toPos[1],size)
    if fromY < toY: return ShipAction.NORTH
    if fromY > toY: return ShipAction.SOUTH
    if fromX < toX: return ShipAction.EAST
    if fromX > toX: return ShipAction.WEST
    
def isShipNearShipyard(ship, me, size):
    too_close = True
    fromX = ship.position[0]
    fromY = ship.position[1]
    print("FromX: %s, FromY: %s" % (fromX, fromY))
    for shipyard in me.shipyards:
        toX = shipyard.position[0]
        toY = shipyard.position[1]
        print("ToX: %s, ToY: %s" % (toX, toY))
        if (abs(fromY - toY) < 2) and (abs(fromX - toX) < 2):
            too_close = False
    return too_close

# Directions a ship can move
directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]

# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard
ship_states = {}
ship_skips = {}

# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    size = config.size
    board = Board(obs, config)
    me = board.current_player

    # If there are no ships, use first shipyard to spawn a ship.
    if len(me.ships) == 0 and len(me.shipyards) > 0:
        me.shipyards[0].next_action = ShipyardAction.SPAWN

    # If there are no shipyards, convert first ship into shipyard.
    if len(me.shipyards) == 0 and len(me.ships) > 0:
        me.ships[0].next_action = ShipAction.CONVERT
    
    # If there is enough halite, spawn a new ship.
    if len(me.shipyards) > 0 and me.halite > 1000:
        print("spawning new ship")
        me.shipyards[len(me.shipyards) - 1].next_action = ShipyardAction.SPAWN
    
    ship_converts = 0
    
    for ship in me.ships:
        ship_skips.setdefault(ship.id, 0)
        
        if ship.next_action == None:
            
            ### Part 1: Set the ship's state 
            if ship.halite < 200: # If cargo is too low, collect halite
                ship_states[ship.id] = "COLLECT"
            if ship.halite > 500: # If cargo gets very big, deposit halite
                ship_states[ship.id] = "DEPOSIT"
                
            # If there is enough halite, and the ship is far enough away, convert it into
            # a base.
            test = isShipNearShipyard(ship, me, size)
            if len(me.shipyards) < 5 and me.halite > 1500 and isShipNearShipyard(ship, me, size) and ship_converts < 2:
                print("converting ship to shipyard")
                ship.next_action = ShipAction.CONVERT
                ship_converts = ship_converts + 1

            ### Part 2: Use the ship's state to select an action
            if ship_states[ship.id] == "COLLECT":
                # If halite at current location running low, 
                # move to the adjacent square containing the most halite
                if ship.cell.halite < 100 and ship_skips[ship.id] > 2:
                    ship_skips[ship.id] = 0
                    neighbors = [ship.cell.north.halite, ship.cell.east.halite, 
                                 ship.cell.south.halite, ship.cell.west.halite]
                    best = max(range(len(neighbors)), key=neighbors.__getitem__)
                    ship.next_action = directions[best]
                    
                    # Don't run into any ships.
                    # This could be clean
                    
                elif ship.cell.halite < 100:
                    ship.next_action = directions[randrange(4)]
                    ship_skips[ship.id] = ship_skips[ship.id] + 1
            if ship_states[ship.id] == "DEPOSIT":
                # Move towards shipyard to deposit cargo
                direction = getDirTo(ship.position, me.shipyards[0].position, size)
                if direction: ship.next_action = direction
                
    return me.next_actions

In [ ]:
from kaggle_environments import make
env = make("halite", debug=True)
env.run(["submission.py", "random", "random", "random"])
env.render(mode="ipython", width=800, height=600)